# <span style="color:red">#</span><span style="color:blue">HashMappers</span><span style="color:red">#</span>
## Live Geoanalysis of Hashtag Hijacking

In [28]:
'''
Login credentials
'''
PORTAL_ADDR = "http://ps0002022.esri.com/portal/home/webmap/viewer.html?useExisting=1&layers=d9974760af6a46059e9ff081324480a4"
USER = "admin"
PW = "esri.agp"

In [29]:
from IPython.display import display
from arcgis import GIS
import os

# login
gis = GIS(PORTAL_ADDR, USER, PW)

In [30]:
'''
Portal properties
'''
print(gis.properties.isPortal)
print(gis.properties.portalMode)
print(gis.properties.portalName)
print(gis.properties.availableCredits)
print(gis.properties.access)
print(gis.properties.canSignInArcGIS)

True
singletenant
ArcGIS Enterprise
0
public
True



# Publish all the service definition files in a folder

The sample below lists all the service definition (.sd) files in a data directory and publishes them as web layers. To publish a service definition file, we first add the .sd file to the Portal, and then call the publish() method:

In [31]:
import json
from arcgis import features
import pandas as pd

In [32]:
# path relative to this notebook
data_dir = "..\\data\\"

#Get list of all files
file_list = os.listdir(data_dir)

#Filter and get only .sd files
json_file_list = [x for x in file_list if x.endswith(".json")]
print("Number of .json files found: " + str(len(json_file_list)))

Number of .json files found: 2


In [33]:
# TODO: Add loop for multiple files
jsonPath = data_dir + json_file_list[1]
tweets_df = pd.read_json(jsonPath)
# tweets_df.head()

tweets_df2 = tweets_df['data'].iloc[0]
tweets_df3 = pd.DataFrame(tweets_df2)
tweets_df3

,user_id,features
geo_type,441857077,User location
id,441857077,441857077
location,441857077,manila
name,441857077,masha
primary_geo,441857077,manila
screen_name,441857077,mawyelly
tweets,441857077,1


In [34]:
# add the initial csv file and publish that as a web layer
item_prop = {'title':'Tweets'}
json_item = gis.content.add(item_properties=item_prop, data="..\\data\\tweets_geo-json.csv")


Item 'tweets_geo-json.csv' already exists.


RuntimeError: Item 'tweets_geo-json.csv' already exists.
(Error Code: 409)

In [36]:
published_item = json_item.publish()
display(published_item)

RuntimeError: Service name 'Tweets' already exists for '0123456789ABCDEF'

In [ ]:
# Loop through each file and publish it as a service
for current_sd_file in sd_file_list:
    item = gis.content.add({}, data_dir + current_sd_file)   # .sd file is uploaded and a .sd file item is created
    published_item = item.publish()                          # .sd file item is published and a web layer item is created
    display(published_item)

In the example shown above, one .sd file produced a web feature layer and another produced a web tile layer

# Publish a feature service from a shapefile and update the item information

To publish a shapefile, we first add the zipped shapefile to the Portal as an item, then call publish() method on the item to create a web layer. Often times, your shape files or service definitions may not contain the metadata you want to show on the portal item. This sample demonstrates how you can update those properties after publishing a web layer.

In [ ]:
data = ".\data\power_pedestals_2012.zip"
shpfile = gis.content.add({}, data)

In [ ]:
shpfile

In [ ]:
published_service = shpfile.publish()

In [ ]:
display(published_service)

The web layer item has minimal information and a default thumbnail.

### Update the layer item's metadata
To update the metadata and set the thumbnail, use the update() method on the web layer's item obtained during publishing.

In [ ]:
thumbnail_path = ".\data\power_pedestals_thumbnail.PNG"
item_properties = {"snippet":"""This dataset was collected from Utah DOT open data portal.
                            Source URL: <a href="http://udot.uplan.opendata.arcgis.com/
                            datasets/a627bb128ac44767832402f7f9bde909_10">http://udot.uplan.opendata.arcgis.com/
                            datasets/a627bb128ac44767832402f7f9bde909_10</a>""",
                   "title":"Locations of power pedestals collected in 2012",
                   "tags":"opendata"}

In [ ]:
published_service.update(item_properties, thumbnail=thumbnail_path)
display(published_service)

# Publish a CSV file and move it into a folder

To publish a CSV file, we first add the .csv file to the Portal, and then call the publish() method to publish it as a layer. Once published, we create a destination folder on the server and then move the published items into that folder

In [ ]:
csv_file = '.\data\Chennai_precipitation.csv'
csv_item = gis.content.add({}, csv_file)

In [ ]:
display(csv_item)

The csv file used in this sample has a column titled `LOCATION` containing place names in text. During the publishing process we specify this column as an `address_fields` parameter. The server geocodes these locations to create point features for the web layer.

In [ ]:
csv_lyr = csv_item.publish(None, {"Address":"LOCATION"})

In [ ]:
display(csv_lyr)

### Create a new folder for the items
The `create_folder()` from `GIS.content` can be used to create a new folder. Once created, the `move()` of the `Item` can be used to move the items into the folder.

In [ ]:
# create a new folder called 'Rainfall Data'
new_folder_details = gis.content.create_folder("Rainfall Data")
print(new_folder_details)

In [ ]:
# move both the csv_item and csv_lyr items into this new folder
csv_item.move(new_folder_details)  # Here you could either pass name of the folder or the dictionary
                                   # returned from create_folder() or folders property on a User object

csv_lyr.move(new_folder_details)

Now that the items are moved, we can request for the item's `ownerFolder` property and ensure it matches the `id` of the folder created in the previous step

In [ ]:
print(csv_lyr.ownerFolder)